In [ ]:
!pip install textstat

In [13]:
from create_dataframe import prepare_legislatives_dataset_lisibilite

csv_p = "/home/onyxia/work/NLP_rendus/.ipynb_checkpoints/data/legislatives_1981.csv"
txt_p = "/home/onyxia/work/NLP_rendus/.ipynb_checkpoints/data/legislatives_1981"

df_final = prepare_legislatives_dataset_lisibilite(csv_p, txt_p)

print("------------Df created-----------")



ImportError: cannot import name 'prepare_legislatives_dataset_lisibilite' from 'create_dataframe' (/home/onyxia/work/NLP_rendus/create_dataframe.py)

In [ ]:
import textstat

# On configure textstat pour le français
textstat.set_lang("fr")

def compute_readability_metrics(df, text_col_raw):
    """
    Calcule les scores de lisibilité sur le texte BRUT (ou peu nettoyé).
    """
    # Application du nettoyage léger
    df['text_clean_light'] = df[text_col_raw].apply(clean_for_readability)
    
    # 1. Score de Facilité de Lecture (Flesch Reading Ease)
    # Plus le score est élevé (vers 100), plus c'est facile à lire.
    df['flesch_score'] = df['text_clean_light'].apply(textstat.flesch_reading_ease)
    
    # 2. Niveau scolaire (Flesch-Kincaid Grade)
    # Indique le nombre d'années d'études nécessaires (système US, 8 = 4ème/3ème).
    df['flesch_kincaid_grade'] = df['text_clean_light'].apply(textstat.flesch_kincaid_grade)
    
    # 3. Indice de complexité (Gunning Fog)
    df['gunning_fog'] = df['text_clean_light'].apply(textstat.gunning_fog)

    return df

# --- Synthèse par parti ---
def summarize_readability_by_party(df, party_col='titulaire-soutien'):
    summary = df.groupby(party_col).agg({
        'flesch_score': 'mean',
        'flesch_kincaid_grade': 'mean',
        'gunning_fog': 'mean'
    }).sort_values('flesch_score', ascending=False) # Du plus simple au plus complexe
    
    return summary

def compare_readability_means(df1, df2, name_df1="Dataset_1", name_df2="Dataset_2"):
    """
    Compare average readability scores between two dataframes.

    INPUTS:
    - df1 : pd.DataFrame
        First dataframe containing readability columns:
        'flesch_score', 'flesch_kincaid_grade', 'gunning_fog'
    - df2 : pd.DataFrame
        Second dataframe containing the same readability columns.
    - name_df1 : str (optional)
        Name used to label the first dataset in the output table.
    - name_df2 : str (optional)
        Name used to label the second dataset in the output table.

    OUTPUT:
    - pd.DataFrame
        Table containing mean readability scores for each dataset.
    """

    # Compute means
    means_df1 = df1[['flesch_score', 
                     'flesch_kincaid_grade', 
                     'gunning_fog']].mean()

    means_df2 = df2[['flesch_score', 
                     'flesch_kincaid_grade', 
                     'gunning_fog']].mean()

    # Create comparison table
    comparison_table = pd.DataFrame({
        name_df1: means_df1,
        name_df2: means_df2
    }).T

    return comparison_table





In [18]:
# 1. On calcule d'abord les scores (cette étape peut prendre 10-30 secondes)
# On utilise 'text' (la colonne brute) car la lisibilité a besoin de la ponctuation
df_final_sans_cat_mixte = compute_readability_metrics(df_final_sans_cat_mixte, 'text_content')

# 2. Maintenant on génère la synthèse
df_readability_summary = summarize_readability_by_party(df_final_sans_cat_mixte)

# 3. On affiche le résultat
display(df_readability_summary)

,flesch_score,flesch_kincaid_grade,gunning_fog
titulaire-soutien,,,
Ligue communiste révolutionnaire,75.530023,10.067589,13.570304
Lutte ouvrière,70.453740,11.925532,15.687017
Parti communiste français,64.590449,12.431989,16.133148
Parti socialiste unifié,64.026146,12.208245,16.050561
ALL_PARTIES,63.667325,12.517685,16.329094
Union pour la démocratie française,61.926172,12.595134,16.391175
Rassemblement pour la République,61.658594,12.713065,16.640100
Front national,61.355593,12.210627,16.482626
Parti socialiste,60.356448,13.051128,16.916893


In [5]:
import re
import textstat

# On s'assure que textstat utilise les normes françaises
textstat.set_lang("fr")

def clean_for_readability(text):
    if not isinstance(text, str):
        return ""
    # 1. Enlever les sauts de ligne bruyants
    text = re.sub(r'\n+', ' ', text)
    # 2. Garder uniquement lettres, chiffres et ponctuation de base (. , ! ? ;)
    # On retire les symboles bizarres type @, #, *, [ ] qui perturbent textstat
    text = re.sub(r'[^\w\s\.,;!?\-]', '', text)
    # 3. Supprimer les espaces multiples
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Appliquer sur ton texte ORIGINAL (pas sur le texte déjà filtré)
df_final_sans_cat_mixte['text_readability'] = df_final_sans_cat_mixte['text'].apply(clean_for_readability)

NameError: name 'df_final_sans_cat_mixte' is not defined

In [ ]:
# Score de Flesch (Facilité de lecture)
df_final_sans_cat_mixte['flesch_ease'] = df_final_sans_cat_mixte['text_readability'].apply(textstat.flesch_reading_ease)

# Score de Gunning Fog (Complexité)
df_final_sans_cat_mixte['gunning_fog'] = df_final_sans_cat_mixte['text_readability'].apply(textstat.gunning_fog)

# Nombre de mots (pour vérifier si un texte est trop court pour être fiable)
df_final_sans_cat_mixte['word_count'] = df_final_sans_cat_mixte['text_readability'].apply(textstat.lexicon_count)